<a href="https://colab.research.google.com/github/sira4154/AI4BA/blob/main/Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.

#Preprocessing

In [2]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
movies_df = pd.read_csv("/content/drive/MyDrive/movie/movies.csv")
ratings_df = pd.read_csv("/content/drive/MyDrive/movie/ratings.csv")

In [5]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies_df.shape

(58098, 3)

In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [8]:
ratings_df.shape

(27753444, 4)

So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy extract function that Pandas has.

In [9]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


Let's look at the result!

In [10]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


###Drop Genres column 

In [11]:
movies_df = movies_df.drop('genres', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


##Check years

In [12]:
s = movies_df["year"].value_counts() #sort_year
s.sort_index()

1874       1
1878       1
1883       1
1887       1
1888       4
        ... 
2014    2297
2015    2312
2016    2219
2017    1891
2018     861
Name: year, Length: 134, dtype: int64

Check duplicate and drop

In [13]:
duplicate = movies_df[movies_df.duplicated(["title"], keep = False)]
duplicate.sort_values(by = ["title"])

,movieId,title,year
29798,128878,1,2014
24351,113214,1,2013
15321,77846,12 Angry Men,1997
1178,1203,12 Angry Men,1957
35224,141816,12 Chairs,1976
...,...,...,...
11910,53154,Zoo,2007
41429,156553,Zoom,2015
11241,47384,Zoom,2006
5801,5899,Zulu,1964


In [14]:
movies_df = movies_df.drop_duplicates(subset = ["title"], keep = "last")

In [15]:
movies_df.shape

(54635, 3)

In [16]:
movies_df.duplicated(["title"]).sum()

0

Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

###Drop timestamp

In [17]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [18]:
ratings_df.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [19]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


##Collaborative Filtering

Now, time to start our work on recommendation systems. 

The first technique we're going to take a look at is called **Collaborative Filtering**, which is also known as **User-User Filtering**. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the **Pearson Correlation Function**.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

The process for creating a User Based recommendation system is as follows:

-   Select a user with the movies the user has watched
-   Based on his rating to movies, find the top X neighbours 
-   Get the watched movie record of the user for each neighbour.
-   Calculate a similarity score using some formula
-   Recommend the items with the highest score

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [175]:
userInput = [
            {'title':'Iron Man 3', 'rating':4},
            {'title':'Jumanji: Welcome to the Jungle', 'rating':4},
            {'title':'Spider-Man', 'rating':4.5},
            {'title':"Star Wars: The Last Jedi", 'rating':5},
            {'title':'John Wick', 'rating':4.5},
            {'title':'Scream 3', 'rating':1.5},
            {'title':'Annabelle', 'rating':1},
            {'title':'Thor: Ragnarok', 'rating':4.5},
            {'title':'Captain America: Civil War', 'rating':4},
            {'title':'Jurassic World', 'rating':4.5},
            {'title':'Forrest Gump', 'rating': 5},
            {'title':'Saving Private Ryan', 'rating':4.5},
            {'title':'The Conjuring 2', 'rating':2},
            {'title':'The Meg', 'rating':1.5},
            {'title':'Ant-Man and the Wasp', 'rating':4},
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Iron Man 3,4.0
1,Jumanji: Welcome to the Jungle,4.0
2,Spider-Man,4.5
3,Star Wars: The Last Jedi,5.0
4,John Wick,4.5
5,Scream 3,1.5
6,Annabelle,1.0
7,Thor: Ragnarok,4.5
8,Captain America: Civil War,4.0
9,Jurassic World,4.5


##Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.



In [176]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,movieId,title,rating
0,356,Forrest Gump,5.0
1,2028,Saving Private Ryan,4.5
2,3273,Scream 3,1.5
3,5349,Spider-Man,4.5
4,102125,Iron Man 3,4.0
5,114713,Annabelle,1.0
6,115149,John Wick,4.5
7,117529,Jurassic World,4.5
8,122916,Thor: Ragnarok,4.5
9,122920,Captain America: Civil War,4.0


##The users who has seen the same movies
Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [177]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
40,3,2028,5.0
122,4,356,3.5
325,4,2028,5.0
604,4,5349,5.0
1122,14,356,4.0


In [178]:
userSubset.shape

(224119, 3)

We now group up the rows by user ID.

In [179]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])
userSubsetGroup.head()

,userId,movieId,rating
40,3,2028,5.0
122,4,356,3.5
325,4,2028,5.0
604,4,5349,5.0
1122,14,356,4.0
...,...,...,...
27752831,283220,356,5.0
27752857,283221,2028,4.0
27752942,283222,5349,2.5
27753002,283224,356,4.0


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [180]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [181]:
userSubsetGroup[100]

(54136,          userId  movieId  rating
 5258627   54136      356     4.0
 5258682   54136     2028     4.0
 5258704   54136     3273     2.5
 5258750   54136     5349     3.5
 5258990   54136   102125     3.5
 5259027   54136   115149     3.5
 5259034   54136   117529     3.5
 5259044   54136   122916     4.0
 5259046   54136   122920     3.5
 5259116   54136   179401     3.0
 5259118   54136   179819     3.5)

#### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the **Pearson Correlation Coefficient**. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

In [182]:
userSubsetGroup = userSubsetGroup[0:10000]

In [183]:
userSubsetGroup[0:2]

[(84278,          userId  movieId  rating
  8186922   84278      356     4.0
  8187056   84278     2028     3.5
  8187131   84278     3273     3.0
  8187241   84278     5349     4.5
  8187973   84278   102125     5.0
  8188087   84278   114713     1.0
  8188095   84278   115149     4.0
  8188114   84278   117529     4.5
  8188143   84278   122916     4.0
  8188145   84278   122920     3.5
  8188303   84278   159858     3.5
  8188484   84278   179401     3.5
  8188486   84278   179819     1.0
  8188533   84278   188301     3.0), (94843,          userId  movieId  rating
  9191825   94843      356     3.0
  9192750   94843     2028     0.5
  9193338   94843     3273     0.5
  9193941   94843     5349     0.5
  9195746   94843   102125     0.5
  9195991   94843   114713     1.0
  9195997   94843   115149     4.0
  9196028   94843   117529     0.5
  9196081   94843   122916     3.0
  9196083   94843   122920     0.5
  9196432   94843   159858     0.5
  9196624   94843   179401     0.5
  919

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient

In [184]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient คำนวณ
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    #ให้ x คือ 
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [185]:
temp_df

,movieId,title,rating
0,356,Forrest Gump,5.0
1,2028,Saving Private Ryan,4.5
4,102125,Iron Man 3,4.0
6,115149,John Wick,4.5
7,117529,Jurassic World,4.5


###calculation example

In [186]:
len(temp_df)

5

In [187]:
tempRatingList

[5.0, 4.5, 4.0, 4.5, 4.5]

In [188]:
tempGroupList

[4.5, 5.0, 2.0, 3.0, 1.5]

In [189]:
Sxx

0.5

In [190]:
Syy

9.299999999999997

In [191]:
Sxy

1.25

In [192]:
nRatings

3

In [193]:
axx = ((1.5**2) + (4.5 ** 2))-((1.5+4.5)**2)/2
print(axx)

4.5


In [194]:
ayy = ((2**2) + (4 ** 2))-((2+4)**2)/2
print(ayy)

2.0


In [195]:
axy = ((1.5*2) + (4.5*4)) - ((1.5+4.5) * (2+4))/2
print(axy)

3.0


In [196]:
p_corr = axy / sqrt(axx*ayy)
print(p_corr)

1.0


###result similarity 

In [197]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF)) # reset index
pearsonDF.head()

,similarityIndex,userId
0,0.405934,84278
1,0.289295,94843
2,0.724242,128610
3,0.534188,220709
4,-0.017260,236068


In [198]:
pearsonDF.tail()

,similarityIndex,userId
4715,-0.655610,282098
4716,0.283981,282632
4717,-0.071429,282670
4718,0.466252,283164
4719,0.579674,283183


##The top x similar users to input user
Now let's get the top 50 users that are most similar to the input.

In [199]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
4600,1.0,261895
2749,1.0,207647
4577,1.0,258192
3573,1.0,88312
3943,1.0,152828


In [200]:
topUsers.tail()

,similarityIndex,userId
2593,0.964901,171283
2968,0.964461,265401
4645,0.964286,270385
621,0.964115,218125
380,0.963558,8446


##Rating of selected users to all movies
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our pearsonDF from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [201]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head(659)

,similarityIndex,userId,movieId,rating
0,1.0,261895,1,4.5
1,1.0,261895,32,4.0
2,1.0,261895,34,3.5
3,1.0,261895,47,4.0
4,1.0,261895,50,3.5
...,...,...,...,...
654,1.0,207647,1784,4.0
655,1.0,207647,1801,3.5
656,1.0,207647,1884,3.5
657,1.0,207647,1907,3.5


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [202]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,261895,1,4.5,4.5
1,1.0,261895,32,4.0,4.0
2,1.0,261895,34,3.5,3.5
3,1.0,261895,47,4.0,4.0
4,1.0,261895,50,3.5,3.5


In [203]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,41.155839,170.055915
2,29.365058,103.207991
3,10.786787,26.514502
4,0.966056,0.966056
5,8.783017,23.428734


In [204]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,4.132000,1
2,3.514653,2
3,2.458054,3
4,1.000000,4
5,2.667504,5


Now let's sort it and see the top 20 movies that the algorithm recommended!

In [205]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
116989,5.0,116989
6408,5.0,6408
60135,5.0,60135
6612,5.0,6612
111778,5.0,111778
116136,5.0,116136
121145,5.0,121145
27002,5.0,27002
84952,5.0,84952


In [206]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
6301,6408,Animals are Beautiful People,1974
12794,60135,Surfwise,2007
16896,84952,Confessions (Kokuhaku),2010
25195,116136,Olive Kitteridge,2014
25542,116989,Tales of the Grim Sleeper,2014
26783,121145,McCullin,2012
48166,171495,Cosmos,NaN


In [207]:
movie_df2 = recommendation_df.rename(columns={recommendation_df.columns[1]: 'id'})

In [208]:
movie_recom =movie_df2.merge(movies_df, left_on='id', right_on="movieId", how='inner')
movie_recom.head(20)

,weighted average recommendation score,id,movieId,title,year
0,5.0,116989,116989,Tales of the Grim Sleeper,2014
1,5.0,6408,6408,Animals are Beautiful People,1974
2,5.0,60135,60135,Surfwise,2007
3,5.0,116136,116136,Olive Kitteridge,2014
4,5.0,121145,121145,McCullin,2012
5,5.0,84952,84952,Confessions (Kokuhaku),2010
6,5.0,171495,171495,Cosmos,NaN
7,5.0,61913,61913,Africa addio,1966
8,5.0,70465,70465,Jerusalema (Gangster's Paradise: Jerusalema),2008
9,5.0,7742,7742,Baxter,1989


In [209]:
movie_recom["year"] = pd.to_numeric(movie_recom['year'])
top10 = movie_recom[movie_recom["year"] >= 2015]
top10.head(10)

,weighted average recommendation score,id,movieId,title,year
16,5.0,141718,141718,Deathgasm,2015.0
21,5.0,147831,147831,Anthony Jeselnik: Thoughts and Prayers,2015.0
25,5.0,155846,155846,War on Everyone,2016.0
60,5.0,173253,173253,Vir Das: Abroad Understanding,2017.0
61,5.0,173467,173467,Sarah Silverman: A Speck of Dust,2017.0
74,5.0,181139,181139,Lynne Koplitz: Hormonal Beast,2017.0
77,5.0,183869,183869,Hereditary,2018.0
82,5.0,187517,187517,Michelle Wolf: Nice Lady,2017.0
83,5.0,188773,188773,Shoplifters,2018.0
88,5.0,161127,161127,The Infiltrator,2016.0


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages

-   Takes other user's ratings into consideration
-   Doesn't need to study or extract information from the recommended item
-   Adapts to the user's interests which might change over time

##### Disadvantages

-   Approximation function can be slow
-   There might be a low of amount of users to approximate
-   Privacy issues when trying to learn the user's preferences
